In [1]:
from Modules.System import *
import symengine as se
import sympy

se.init_printing(pretty_print=True)

In [2]:
iG = se.Symbol("i_G" , real = True)
c = se.Symbol("c" , real = True)
s = se.Symbol("s" , real = True)
m = se.Symbol("m" , real = True)
CA = se.Symbol("C_A" , real = True)
CM = se.Symbol("C_M" , real = True)
km = se.Symbol("k_m" , real = True)
RA = se.Symbol("R_A" , real = True)

params = [iG,c,s,m,CA,CM,km,RA]

In [3]:
q_var = sys_var(["qM","qA"],2,2,True)
u_var = sys_var(["M_{Mot}"],1,0,True)
System_object.init_diff_t(q_var,u_var)
u = u_var.vars
display(q_var.vars)
[q,q_dot,q_ddot] = q_var.vars
[[qM,qA],[qM_dot, qA_dot],[qM_ddot, qA_ddot]] = q_var.vars
display(u)
M_Mot = u[0]

[[qM(t)]
 [qA(t)],
 [\dot{qM}(t)]
 [\dot{qA}(t)],
 [\ddot{qM}(t)]
 [\ddot{qA}(t)]]

[M_{Mot}(t)]

In [4]:
K_J_AS = se.zeros(3)
K_J_AS[2,2] = CA

K_J_AS

[0, 0, 0]
[0, 0, 0]
[0, 0, C_A]

In [5]:
K_J_MS = se.zeros(3)
K_J_MS[2,2] = CM


K_J_MS

[0, 0, 0]
[0, 0, 0]
[0, 0, C_M]

In [6]:
R_IK = Drehmatrix(se.Matrix([0,0,qM]))
display(R_IK)

[cos(qM(t)), -sin(qM(t)), 0]
[sin(qM(t)), cos(qM(t)), 0]
[0, 0, 1]

In [7]:
K_r_s = se.Matrix([s,0,0])
K_r_s

[s]
[0]
[0]

In [8]:
K_w_K = se.Matrix([0,0,qA_dot+qM_dot])
K_w_K

[0]
[0]
[\dot{qA}(t) + \dot{qM}(t)]

In [9]:
K_w_M = se.Matrix([0,0,iG*qM_dot])
K_w_M

[0]
[0]
[i_G*\dot{qM}(t)]

In [10]:
K_v_S = diff_t(K_r_s) + K_w_K.cross(K_r_s)
K_v_S

[0]
[(\dot{qA}(t) + \dot{qM}(t))*s]
[0]

In [11]:
K_p = m * K_v_S
K_p

[0]
[(\dot{qA}(t) + \dot{qM}(t))*m*s]
[0]

In [12]:
K_L_A = K_J_AS * K_w_K
K_L_A

[0]
[0]
[(\dot{qA}(t) + \dot{qM}(t))*C_A]

In [13]:
K_L_M = K_J_MS * K_w_M
K_L_M

[0]
[0]
[i_G*C_M*\dot{qM}(t)]

In [14]:
K_Me_A = se.Matrix([0,0,-qA * c])
K_Me_A

[0]
[0]
[-c*qA(t)]

In [15]:
K_Me_M = se.Matrix([0,0,M_Mot]) - K_Me_A / iG
K_Me_M

[0]
[0]
[c*qA(t)/i_G + M_{Mot}(t)]

In [16]:
part1 = K_v_S.jacobian(q_dot).transpose() * (diff_t(K_p) + K_w_K.cross(K_p))


part1

[(\ddot{qA}(t) + \ddot{qM}(t))*m*s**2]
[(\ddot{qA}(t) + \ddot{qM}(t))*m*s**2]

In [17]:
part2 = K_w_K.jacobian(q_dot).transpose() * (diff_t(K_L_A) + K_w_K.cross(K_L_A) - K_Me_A)
part2

[c*qA(t) + (\ddot{qA}(t) + \ddot{qM}(t))*C_A]
[c*qA(t) + (\ddot{qA}(t) + \ddot{qM}(t))*C_A]

In [18]:
part3 = K_w_M.jacobian(q_dot).transpose() * (diff_t(K_L_M) + K_w_M.cross(K_L_M) - K_Me_M)
part3

[i_G*(i_G*C_M*\ddot{qM}(t) - (c*qA(t)/i_G + M_{Mot}(t)))]
[0]

In [19]:
parts = part1 + part2 + part3
parts

[c*qA(t) + i_G*(i_G*C_M*\ddot{qM}(t) - (c*qA(t)/i_G + M_{Mot}(t))) + (\ddot{qA}(t) + \ddot{qM}(t))*C_A + (\ddot{qA}(t) + \ddot{qM}(t))*m*s**2]
[c*qA(t) + (\ddot{qA}(t) + \ddot{qM}(t))*C_A + (\ddot{qA}(t) + \ddot{qM}(t))*m*s**2]

In [20]:
M = parts.jacobian(q_ddot)
M

[C_A + i_G**2*C_M + m*s**2, C_A + m*s**2]
[C_A + m*s**2, C_A + m*s**2]

In [21]:
g = parts - M @  q_ddot
g

[c*qA(t) + i_G*(i_G*C_M*\ddot{qM}(t) - (c*qA(t)/i_G + M_{Mot}(t))) + (\ddot{qA}(t) + \ddot{qM}(t))*C_A + (\ddot{qA}(t) + \ddot{qM}(t))*m*s**2 - (\ddot{qA}(t)*(C_A + m*s**2) + \ddot{qM}(t)*(C_A + i_G**2*C_M + m*s**2))]
[c*qA(t) + (\ddot{qA}(t) + \ddot{qM}(t))*C_A + (\ddot{qA}(t) + \ddot{qM}(t))*m*s**2 - (\ddot{qA}(t)*(C_A + m*s**2) + \ddot{qM}(t)*(C_A + m*s**2))]

In [22]:
f = q_dot.col_join(-M.inv() * g)
sympy.simplify(f)

Matrix([
[                                                                                                                  \dot{qM}(t)],
[                                                                                                                  \dot{qA}(t)],
[                                                                                      (c*qA(t) + i_G*M_{Mot}(t))/(C_M*i_G**2)],
[(-C_A*c*qA(t) - C_A*i_G*M_{Mot}(t) - C_M*c*i_G**2*qA(t) - c*m*s**2*qA(t) - i_G*m*s**2*M_{Mot}(t))/(C_M*i_G**2*(C_A + m*s**2))]])

In [23]:
h = q.col_join(q_dot)
sys = System_object(f,h,q_var,u_var,params,False)

In [24]:
[A,B,C,D] = sys.linearize(se.Matrix([0,0,0,0]),se.Matrix([0]),True)
display(A,B,C,D)

[0, 0, 1, 0]
[0, 0, 0, 1]
[0, c/(i_G**2*C_M), 0, 0]
[0, -c*(C_A + i_G**2*C_M + m*s**2)/(i_G**2*C_M*(C_A + m*s**2)), 0, 0]

[0]
[0]
[1/(i_G*C_M)]
[-1/(i_G*C_M)]

[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]

[0]
[0]
[0]
[0]

In [25]:
sys.write_ABCD_to_file("ss_matrices")
sys.write_s_funs("Test")

In [26]:

f1,f2 = se.cse(f)
display(f1, f1[0])
display(f2, f2[2])

[(x0, m*s**2),
 (x1, C_A + x0),
 (x2, x1 + i_G**2*C_M),
 (x3, x2**(-1)),
 (x4, x1**2*x3),
 (x5, (x1 - x4)**(-1)),
 (x6, \ddot{qM}(t)),
 (x7, \ddot{qA}(t)),
 (x8, x1*x7),
 (x9, c*qA(t)),
 (x10, x6 + x7),
 (x11, x9 + x0*x10 + x10*C_A),
 (x12, x3*(x11 + i_G*(x6*i_G*C_M - (x9/i_G + M_{Mot}(t))) - (x8 + x2*x6))),
 (x13, x5*(x11 - (x8 + x1*x6)))]

(x0, m*s**2)

[\dot{qM}(t), \dot{qA}(t), -x12*(1 + x4*x5) + x1*x3*x13, -x13 + x1*x5*x12]

-x12*(1 + x4*x5) + x1*x3*x13

In [27]:
#for temp in f1:
#    display(temp[0] +" = " + temp[1])

In [28]:
x = sympy.Symbol("x")
t = sympy.Symbol("t")
test = sympy.Piecewise((x, t > 5),(-x, t > 7), (x**2, t> 10),(-x * t, t <= 0), (t, True))
display(test)

Piecewise((x, t > 5), (-x, t > 7), (x**2, t > 10), (-t*x, t <= 0), (t, True))

In [29]:
print(sympy.octave_code(test, inline = False))

if (t > 5)
  x
elseif (t > 7)
  -x
elseif (t > 10)
  x.^2
elseif (t <= 0)
  -t.*x
else
  t
end
